## Define Helper Function

In [1]:
assignments = []
rows = 'ABCDEFGHI'
cols = '123456789'

def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [a+b for a in A for b in B]

boxes = cross(rows, cols)

row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]

diagonal_units = [[r+c for r,c in zip(rows,cols)], [r+c for r,c in zip(rows,cols[::-1])]]


In [2]:
diagonal = True
if diagonal:
    unitlist = row_units + column_units + square_units + diagonal_units
else:
    unitlist = row_units + column_units + square_units

units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

### 使用 assign_value 去更新values字典

In [2]:
def assign_value(values, box, value):
    """
    Please use this function to update your values dictionary!
    Assigns a value to a given box. If it updates the board record it.
    """
    # Don't waste memory appending actions that don't actually change any values
    if values[box] == value:
        return values

    values[box] = value
    if len(value) == 1:
        assignments.append(values.copy())
    return values

In [ ]:
# Find all instances of naked twins

    # loop over units checking if there is are any naked twins

        # get all boxes with a length of two


        # now get all the naked pair twins ie exists twice in pairs


        # iterate through the naked_twins removing the values from the non naked twin boxes


            # loop over the boxes in the unit and remove the pair from them
            # we could use peers as instead but would still need the check to
            # ensure we aren't impacting the other naked twin

                # test to ensure we don't eliminate the values from either naked-twin

                    # Eliminate the naked twins as possibilities for their peers
                    # the sorted is required for passing the test assertions as the box values
                    # are expected to be in numerical order.

In [3]:
def naked_twins(values):
    """Eliminate values using the naked twins strategy.
    Args:
        values(dict): a dictionary of the form {'box_name': '123456789', ...}
    Returns:
        the values dictionary with the naked twins eliminated from peers.
    """
    # Find all instances of naked twins

    ##  Collect the boxes with two possibility solution
    twins_candidate = [ box for box in values if len(values[box]) == 2 ]

    ##  naked twins pairing
    nakedtwins = [[box_1, box_2] for box_1 in twins_candidate for box_2 in peers[box_1] \
                  if values[box_1]==values[box_2]]

    for i in range(len(nakedtwins)):
        paired_1 = nakedtwins[i][0] #  first box of paired
        paired_2 = nakedtwins[i][1] #  second one which has same element with 1st

        peers_of_1 = set(peers[paired_1]) # Find the peers
        peers_of_2 = set(peers[paired_2]) # Find the peers

        ## peer intesection
        peers_in_common = [ x for x in peers_of_1 if x in peers_of_2]

        for peer_val in peers_in_common:
            if len(values[peer_val])>2: # choose the peers which possible elements over 2 
                for rm in values[paired_2]:
                    values = assign_value(values, peer_val, values[peer_val].replace(rm,''))
    return values


In [4]:
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

In [5]:
def grid_values(grid):
    """
    Convert grid into a dict of {square: char} with '123456789' for empties.
    Args:
        grid(string) - A grid in string form.
    Returns:
        A grid in dictionary form
            Keys: The boxes, e.g., 'A1'
            Values: The value in each box, e.g., '8'. If the box has no value, then the value will be '123456789'.
    """
    chars = []
    digits = '123456789'
    for c in grid:
        if c in digits:
            chars.append(c)
        if c == '.':
            chars.append(digits)
    assert len(chars) == 81
    return dict(zip(boxes, chars))

In [6]:
def eliminate(values):
    solved = [ x for x in boxes if len(values[x])==1]
    for unit in solved:
        digit = values[unit]
        for i in peers[unit]:
            
            values = assign_value(values, i, values[i].replace(digit, ''))
            ## values[i] = values[i].replace(digit,'')
    return values

In [7]:
def only_choice(values):
    for unit in unitlist:
        for digit in '123456789':
            ans = [ x for x in unit if digit in values[x]]
            if len(ans) == 1:
                values = assign_value(values, ans[0], digit)
                ## values[ans[0]] = digit
    return values

In [8]:

def reduce_puzzle(values):
    result = False
    while not result:
        solved_before = len([x for x in values if len(values[x])==1])
        values = eliminate(values)
        values = only_choice(values)
        values = naked_twins(values)
        
        solved_after = len([x for x in values if len(values[x])==1])

        result = (solved_before==solved_after)
        if len([box for box in values if len(values[box]) == 0]):
            return False
    return values


In [9]:
def search(values):

    values = reduce_puzzle(values)

    if values is False:
        return False
    values = reduce_puzzle(values)
    if all(len(values[s])==1 for s in boxes):
        return values

    _, s = min([ (len(values[i]), i) for i in values if len(values[i])>1 ])

    for i in values[s]:
        new_su = values.copy()
        new_su[s] = i

        attempt = search(new_su)
        if attempt:
            return attempt

In [11]:
def solve(grid):
    """
    Find the solution to a Sudoku grid.
    Args:
        grid(string): a string representing a sudoku grid.
            Example: '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    Returns:
        The dictionary representation of the final sudoku grid. False if no solution exists.
    """
    values = grid_values(grid)
    values = search(values)

    return values

if __name__ == '__main__':
    #diag_sudoku_grid = '6...41....1......3.3...2.8...6.....25..1.7..43.....1...4.6...9.2......5....25...6'
    diag_sudoku_grid  = '.275.....1....8....9......25...9281...........7236...59......8....9....3.....574.'

    try:
        unitlist = row_units + column_units + square_units + diag_1 + diag_2
        units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
        peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)
        display(solve(diag_sudoku_grid))
    except:
        unitlist = row_units + column_units + square_units #+ diag_1 + diag_2
        units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
        peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)
        display(solve(diag_sudoku_grid))       
    try:
        from visualize import visualize_assignments
        visualize_assignments(assignments)

    except SystemExit:
        pass
    except:
        print('We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.')


3 2 7 |5 4 9 |1 6 8 
1 6 5 |2 7 8 |9 3 4 
4 9 8 |6 1 3 |5 7 2 
------+------+------
5 3 4 |7 9 2 |8 1 6 
6 1 9 |8 5 4 |3 2 7 
8 7 2 |3 6 1 |4 9 5 
------+------+------
9 5 3 |4 2 7 |6 8 1 
7 4 1 |9 8 6 |2 5 3 
2 8 6 |1 3 5 |7 4 9 
We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.
